# Importes

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import slate3k as slate
import nltk
from lxml import etree
from urllib.request import urlopen

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/p1ntsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Funções e classes

In [3]:
class RequestBeautifulSoup:
    def __init__(self,url):
        self.url = url
        self.intervalos = None
        self.perguntas = None
        self.respostas = None
    
    # Request do soup da biblioteza BeautifulSoup
    def requestSoup(self):
        response = get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    # Intervalos de onde estão as respostas
    def getIntervalos(self,listaPrincipal):
        intervalos = []
        for i in range(len(self.perguntas[1])):
            if(i != len(self.perguntas[1])-1):
                intervalos.append((self.perguntas[1][i]+1,self.perguntas[1][i+1]))
            else:
                intervalos.append((self.perguntas[1][i]+1,len(listaPrincipal)))
        self.intervalos = intervalos

    # Perguntas e suas posições
    def getPerguntas(self,soup, chaves = None):
        perguntas = []
        posi = []
        for i in range(len(soup)):
            if("?" in soup[i].text):
                if(chaves is not None):
                    texto = soup[i].text
                    for j in chaves:
                        texto = texto.replace(j,"")
                    perguntas.append(texto)
                    posi.append(i)
                else:
                    perguntas.append(soup[i].text)
                    posi.append(i)

        self.perguntas = [perguntas,posi]
    
    # Respostas
    def getRespostas(self,soup,chaves = None,subs = ""):
        respostas = []
        for i in self.intervalos:
            text = ""
            for j in range(i[0],i[1]):
                text += soup[j].text
            if(chaves is not None):
                for j in chaves:
                    text = text.replace(j,subs)
                respostas.append(text)
            else:
                respostas.append(text)
        self.respostas = respostas
        
    def getComparacao(self,first,second):
        if(len(first) == len(second)):
            return True
        else:
            return False
        
    def criarDataframePDF(self,perguntas,respostas,salve = None):
        if(salve is not None):    
            pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve + '.csv', index= False)
        else:
            return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})
    

In [40]:


def getXpathWay(url):
    response = urlopen(url)
    htmlparser = etree.HTMLParser()
    return etree.parse(response, htmlparser)
        

def getPerguntasXpath(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
                
    return [perguntas,posi]
                
def getPerguntasPDF(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
    return [perguntas,posi]

def getRespostasPDF(soup,intervalos,chaves = None):
    respostas = []
    for i in intervalos:
        text = ""
        for j in range(i[0],i[1]):
            text += soup[j]
        if(chaves is not None):
            for k in chaves:
                text = text.replace(k,"")
        respostas.append(text)
    return respostas

# Intervalos de onde estão as respostas
def getIntervalos(posicoes,listaPrincipal):
    intervalos = []
    for i in range(len(posicoes)):
        if(i != len(posicoes)-1):
            intervalos.append((posicoes[i]+1,posicoes[i+1]))
        else:
            intervalos.append((posicoes[i]+1,len(listaPrincipal)))
    return intervalos

def criarDataframePDF(perguntas,respostas,salve = None):
    if(salve is not None):    
        pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve + '.csv', index= False)
    else:
        return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})

# Código

In [21]:
with open('./PR.html','r') as f:
    objeto_soup_1 = BeautifulSoup(f, 'html.parser')

In [18]:
objeto_soup = RequestBeautifulSoup('https://vidasaudavel.einstein.br/covid-19-faq/')
soup = objeto_soup.requestSoup()

In [20]:
soup

<!DOCTYPE html>
 <html lang="en"> <head> <meta charset="utf-8"/> <meta content="ie=edge" http-equiv="x-ua-compatible"/> <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/> <title>StackPath</title> <style> *{box-sizing: border-box;}*::before, *::after{box-sizing: border-box;}*:focus{outline: none;}html{-moz-osx-font-smoothing: grayscale;-ms-overflow-style: -ms-autohiding-scrollbar;-webkit-font-smoothing: antialiased;font-size: 16px;}body{background-color: #fff;color: #545963;font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", "Roboto", "Oxygen", "Ubuntu", "Cantarell", "Fira Sans", "Droid Sans", "Helvetica Neue", sans-serif;font-weight: 400;line-height: 1.6;margin: 0;min-width: 20rem;}h1, p{margin: 0;padding: 0;}h1{font-size: 2rem;font-weight: 400;line-height: 1;}h1:not(:last-child){margin-bottom: 3rem;}p{font-size: 1rem;}p:not(:last-child){margin-bottom: 1.5rem;}a{color: #0934a0;text-decoration: none;}a:hover{color: #0934a0;text-decoration: un

In [26]:
#div_soup = objeto_soup_1.find('div', class_= "post_single__content")
list_soup = objeto_soup_1.find_all('div')
#list_soup = div_soup.find_all(['p','ul','il','a','h2','h3','h4'])
#list_soup = objeto_soup_1.find_all(['div'])

In [82]:
dfs = []
for i in ['sp-ea-1885','sp-ea-1886','sp-ea-1887','sp-ea-1888','sp-ea-1889','sp-ea-1896','sp-ea-1961','sp-ea-2061','sp-ea-2062']:
    
    list_soup_um = objeto_soup_1.find('div',id=i)
    list_soup = list_soup_um.find_all(['a','p','ul','li'])
    list_soup = [i.text for i in list_soup]
    perguntas = getPerguntasPDF(list_soup,['\xa0','\n','\t','\r'])
    intervalos = getIntervalos(perguntas[1],list_soup)
    respostas = getRespostasPDF(list_soup,intervalos,['\t','\r'])
    dfs.append(criarDataframePDF(perguntas[0],respostas))

df_result = pd.concat(dfs)

In [86]:
df_result.reset_index(drop=True,inplace=True)

In [87]:
df_result.to_csv('./vidasaudavel.einstein.br.csv',index=False)

In [70]:
list_soup = list_soup_um.find_all(['a','p','ul','li'])

In [71]:
list_soup

[<a class="collapsed" data-sptarget="#collapse18880" data-sptoggle="spcollapse" href=""><i class="ea-expand-icon fa fa-plus"></i> É seguro amamentar durante a infecção por coronavírus?</a>,
 <p>Embora os estudos científicos na área sejam escassos e recentes, ainda não há relatos de transmissão do coronavírus através do leite materno.</p>,
 <p>A contaminação ocorre principalmente por meio de gotículas geradas por uma pessoa contaminada durante tosse/espirros ou contato com secreções. É importante o uso de máscaras e álcool gel (70%) além de discutir com o seu médico a melhor alternativa neste momento tão delicado.</p>,
 <p>Fonte: Centers for Disease Control and Prevention (atualizado 21/02/2020)</p>,
 <a class="collapsed" data-sptarget="#collapse18881" data-sptoggle="spcollapse" href=""><i class="ea-expand-icon fa fa-plus"></i> Posso passar o coronavírus para o meu filho durante o parto?</a>,
 <p>Ainda não conseguimos afirmar sobre o potencial de transmissão vertical do coronavírus (ant

In [8]:
list_soup = list_soup[0].find_all(['div'])

In [173]:
list_soup = list_soup[22:]

In [135]:
text = ""
for i in list_soup:
    #print(i.text)
    text += i.text

In [72]:
list_soup = [i.text for i in list_soup]

In [73]:
list_soup

[' É seguro amamentar durante a infecção por coronavírus?',
 'Embora os estudos científicos na área sejam escassos e recentes, ainda não há relatos de transmissão do coronavírus através do leite materno.',
 'A contaminação ocorre principalmente por meio de gotículas geradas por uma pessoa contaminada durante tosse/espirros ou contato com secreções. É importante o uso de máscaras e álcool gel (70%) além de discutir com o seu médico a melhor alternativa neste momento tão delicado.',
 'Fonte: Centers for Disease Control and Prevention (atualizado 21/02/2020)',
 ' Posso passar o coronavírus para o meu filho durante o parto?',
 'Ainda não conseguimos afirmar sobre o potencial de transmissão vertical do coronavírus (antes, durante ou depois do parto). Dos poucos relatos científicos que existem sobre o coronavírus que causa COVID-19, nenhum dos recém-nascidos de mães doentes apresentaram o vírus - que não foi encontrado no líquido amniótico ou no leite materno.',
 'A contaminação ocorre princ

In [125]:
list_soup = [i for i in text.split('\n') if i != ' ' or i != '']

In [74]:
perguntas = getPerguntasPDF(list_soup,['\xa0','\n','\t','\r'])

In [75]:
intervalos = getIntervalos(perguntas[1],list_soup)

In [76]:
respostas = getRespostasPDF(list_soup,intervalos,['\t','\r'])

In [77]:
len(perguntas[0]) == len(respostas)

True

In [78]:
criarDataframePDF(perguntas[0],respostas)

,Perguntas,Respostas
0,É seguro amamentar durante a infecção por cor...,Embora os estudos científicos na área sejam es...
1,Posso passar o coronavírus para o meu filho d...,Ainda não conseguimos afirmar sobre o potencia...
2,Gestantes podem transmitir o vírus para o bebê?,Ainda não conseguimos afirmar sobre o potencia...
3,Lactante com suspeita da doença pode amamentar?,"As informações são escassas, mas até o momento..."
4,Ouvi sobre casos de recém-nascidos que testar...,Estudos existentes hoje mostram que o vírus nã...
5,Grávidas correm risco maior?,Sim. Embora ainda não existam dados suficiente...
6,Posso mandar meu filho para escola?,"A partir de 13 de março de 2020, o Governo do ..."
7,Quais medidas os pais e responsáveis devem e ...,"Se compatível com a idade, os pais ou responsá..."
8,"Parques, praças, clubes etc. estão liberados ...","Atividades ao ar livre não são proibidas, pois..."
9,O que posso fazer para proteger meu/minha fil...,As crianças devem seguir as mesmas orientações...


In [46]:
df_um = criarDataframePDF(perguntas[0],respostas)

In [57]:
df_dois = criarDataframePDF(perguntas[0],respostas)

In [68]:
df_tres = criarDataframePDF(perguntas[0],respostas)

In [79]:
df_quatro = criarDataframePDF(perguntas[0],respostas)

In [77]:
objeto_soup.getPerguntas(list_soup,['\xa0','\n'])
objeto_soup.getIntervalos(list_soup)
objeto_soup.getRespostas(list_soup,['\xa0','\n'])

In [78]:
print(objeto_soup.perguntas[0])
print(objeto_soup.respostas)
print(objeto_soup.getComparacao(objeto_soup.perguntas[0],objeto_soup.respostas))

['Como prevenir?As orientações do Ministério da Saúde sobre a prevenção de contágio são: lavar as mãos com água e sabão ou usar álcool em gel; ao tossir ou espirrar cobrir o nariz e a boca com um lenço ou braço, não utilizar as mãos; manter ambientes bem ventilados; e não compartilhar objetos pessoais; reduzir o contato social.', 'Quem deve ir ao hospital?Só deve ir ao hospital quem estiver se sentindo muito mal ou apresentar sintomas como febre alta e dificuldade para respirar. Se for só um resfriado ou gripe leve, a recomendação é ficar em casa. De acordo com infectologistas, ir ao hospital, além de sobrecarregar o sistema, aumenta o risco de a pessoa contrair uma infecção.', 'Quais são os sintomas associados a pandemia?Os sintomas do Covid-19 envolvem febre, cansaço e tosse seca. Alguns pacientes podem apresentar dores, congestão nasal, coriza, tosse e diarreia. Enquanto outros pacientes podem ser assintomáticos, ou seja, estarem infectados pelo vírus, mas não apresentarem sintomas.

In [57]:
objeto_soup.criarDataframePDF(objeto_soup.perguntas[0],objeto_soup.respostas,'coronavirus.rj.gov.br')

In [87]:
with open('../img/1.pdf','rb') as f:
    extract = slate.PDF(f)[0]

In [88]:
extract.split('\n\n')

['Algumas pessoas são infectadas mas não desenvolvem sintoma algum e não se sentem mal. A ',
 'maioria das pessoas (cerca de 80%) se recupera da doença sem a necessidade de nenhum ',
 'tratamento especial. Os casos mais graves desenvolvem dificuldade de respirar.  ',
 'Pessoas idosas, aquelas com o sistema imunológico enfraquecido ou problemas de saúde ',
 'preexistentes, como hipertensão, problemas cardíacos ou diabetes, estão mais suscetíveis a ',
 'adoecer gravemente. Quem \natendimento médico. ',
 'tiver ',
 'febre, ',
 'tosse e dificuldade para ',
 'respirar deve procurar ',
 'O COVID-19 é um vírus novo e não se sabe ao certo como interage com Chagas, pois não temos ',
 'evidências científicas suficientes. É certo que aqueles que já têm problemas cardíacos graves ',
 'causados pela doença de Chagas ou um sistema imunológico enfraquecido correm mais risco ',
 'de desenvolver sintomas mais graves de COVID-19. ',
 'Se sou paciente crônico de doença de Chagas sem complicações graves, 

In [84]:
perguntas = ['Quais são os sintomas de COVID-19? São diferentes para um paciente de doença de Chagas?','Se sou paciente crônico de doença de Chagas sem complicações graves, sou mais vulnerável ao COVID-19?','Se estou na fase aguda da doença de Chagas e tenho sintomas de COVID- 19, o que devo fazer?','Se eu tiver sintomas ou o resultado do teste de COVID-19 for positivo, preciso interromper o tratamento de doença de Chagas?','O que devo fazer se eu tiver doença de Chagas sem tratamento específico e tiver sintomas de COVID-19?','O que devo fazer para proteger minha família e a mim do COVID-19 se tenho doença de Chagas?','Se eu tiver complicações cardiológicas ou digestivas causadas pela doença de Chagas, o que preciso fazer para me proteger do COVID-19?','Devo continuar frequentando o centro de saúde para dar continuidade ao tratamento de Chagas durante a epidemia de COVID-19?','Se eu receber um resultado positivo para COVID-19 e tiver doença de Chagas, quais são os tratamentos que não devo tomar?','Se estou grávida e suspeito que tenha contraído doença de Chagas, sou mais vulnerável ao COVID-19?','Se eu estiver grávida, tiver doença de Chagas e receber um resultado positivo para COVID-19, o que devo fazer?']

In [85]:
perguntas

['Quais são os sintomas de COVID-19? São diferentes para um paciente de doença de Chagas?',
 'Se sou paciente crônico de doença de Chagas sem complicações graves, sou mais vulnerável ao COVID-19?',
 'Se estou na fase aguda da doença de Chagas e tenho sintomas de COVID- 19, o que devo fazer?',
 'Se eu tiver sintomas ou o resultado do teste de COVID-19 for positivo, preciso interromper o tratamento de doença de Chagas?',
 'O que devo fazer se eu tiver doença de Chagas sem tratamento específico e tiver sintomas de COVID-19?',
 'O que devo fazer para proteger minha família e a mim do COVID-19 se tenho doença de Chagas?',
 'Se eu tiver complicações cardiológicas ou digestivas causadas pela doença de Chagas, o que preciso fazer para me proteger do COVID-19?',
 'Devo continuar frequentando o centro de saúde para dar continuidade ao tratamento de Chagas durante a epidemia de COVID-19?',
 'Se eu receber um resultado positivo para COVID-19 e tiver doença de Chagas, quais são os tratamentos que n

# Merge dos crawlers

In [89]:
from os import listdir
from os.path import isfile, join
mypath = './Perguntas e Respostas nao submetidas/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [92]:
dfs = []
for i in onlyfiles:
    dfs.append(pd.read_csv('./Perguntas e Respostas nao submetidas/' + i))

In [94]:
result = pd.concat(dfs)
result.shape

(245, 2)

In [96]:
result.reset_index(drop=True,inplace=True)

In [97]:
result

,Perguntas,Respostas
0,O que é coronavírus?,Coronavírus é uma família de vírus que causam ...
1,Como o coronavírus é transmitido?,A transmissão do coronavírus de pessoa para pe...
2,Quais os sintomas?,É um vírus com os sintomas muito semelhantes a...
3,Como é feito o diagnóstico do coronavírus no R...,A Vigilância municipal coleta uma amostra de m...
4,Como é o tratamento do coronavírus?,Não existe tratamento específico para o corona...
...,...,...
240,Tenho um caso confirmado em casa: uso louça e...,"Não é obrigatório o uso de talheres, pratos e ..."
241,Tenho um caso confirmado em casa: como lavar ...,Em máquina de lavar comum. A lavagem no tanque...
242,Tenho um caso confirmado em casa: devo deixa...,"Sim, se possível deixe as janelas sempre abert..."
243,Tenho um caso confirmado em casa: como descar...,"O lixo deve ser descartado como lixo comum, de..."


In [99]:
df_projeto = pd.read_csv('../../DataSets/Projeto_Repositorio/repositorio_projeto.csv')

In [101]:
df_projeto.drop(['Unnamed: 0'],axis=1,inplace=True)
df_projeto.columns = ['Perguntas','Respostas']

In [107]:
aa = pd.concat([df_projeto,result])

In [109]:
aa.reset_index(drop=True,inplace=True)

In [111]:
aa.to_csv('../../DataSets/Projeto_Repositorio/repositorio_v2.csv')